# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()


[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179]]

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data=song_data[0]
song_data=song_data.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()
cur.execute('SELECT * FROM songs')
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', '0', 218.93179)
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', '0', 266.39628)
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', '1984', 269.81832)
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', '0', 189.57016)
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', '1969', 148.03546)
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', '0', 267.7024)
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', '0', 114.78159)
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', '0', 218.77506)
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', '1982', 233.40363)
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', '2007', 209.60608)
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', '0', 177.47546)
('SOUQQEA12A8C134B1B', 'High Tide', 'ARIG6O41187B988BDD', '0', 228.5971

In [10]:
cur.execute("ROLLBACK")

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data = artist_data.tolist()[0]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()
cur.execute("SELECT * FROM artists")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)
('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan)
('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan)
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', nan, nan)
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)
('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan)
('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan)
('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955)
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)
('ARXR32B1187FB57099', 'Gob', '', nan, nan)
('ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan)
('ARIG6O41187B988BDD', 'Richard Souther', 'United States', 37.16793, -95.84502)
('AR1ZHYZ1187FB3C717', 'Faiz Ali Faiz', '', nan, nan)
('ARD842G1187B997376', 'Blue Rodeo', 'Toronto, Ontario, Canada', 43.64856, -79.38533)
('ARL7K851187B99ACD2', 'Andy Andy', '', nan, nan)
('ARHHO3O1187B989413', 'Bob Azzam', '', nan, n

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files("data/log_data")

In [14]:
filepath = log_files[0]

In [15]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [17]:
t = pd.to_datetime(df['ts'], unit='ms')

In [18]:
time_data = []
for line in t:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [19]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)

In [20]:
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,Friday
1,2018-11-30 01:08:41.796,1,30,48,11,2018,Friday
2,2018-11-30 01:12:48.796,1,30,48,11,2018,Friday
3,2018-11-30 01:17:05.796,1,30,48,11,2018,Friday
4,2018-11-30 01:20:56.796,1,30,48,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [24]:
cur.execute("ROLLBACK")

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():
  
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
     
    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'),row.userId, row.level,songid,artistid,row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [26]:
songplay_data

(Timestamp('2018-11-30 19:54:24.796000'),
 '5',
 'free',
 None,
 None,
 985,
 'Detroit-Warren-Dearborn, MI',
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"')

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.